In [248]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from snorkel_process import snorkel_process
import warnings
import numpy as np
from tqdm import tqdm
import scispacy
from scispacy.umls_linking import UmlsEntityLinker
warnings.filterwarnings('ignore')
from snorkel_process_NIH import build_raw_data
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)
from bert_score import score,scorer

from sklearn.linear_model import SGDRegressor

In [2]:
allfile, allsent=build_raw_data('/home/text_download/inhibitor_NIH_ab.csv')

100%|██████████| 3269/3269 [00:02<00:00, 1461.70it/s]


In [3]:
allsent['newpid']=range(len(allsent))

In [4]:
nlp = spacy.load('en_core_sci_lg')
linker = UmlsEntityLinker(resolve_abbreviations=True)
nlp.add_pipe(linker)


In [5]:
with open('/home/text_download/keylist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        keylist=line.split(',')

In [6]:
valuelist = []
with open('/home/text_download/valuelist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        valuelist.append(line.split(','))


In [78]:
def matcher (nlp, terms):
    patterns = [nlp(text) for text in terms]
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add("TerminologyList", None, *patterns)
    return matcher

In [79]:
def reportfu(dataframe, pidlist,matcher,nlp):
    allmatchnu=[]
    for doc in dataframe.sent.values:
        doc=nlp(str(doc))
        matches = matcher(doc)
        allmatch=[]
        for match_id, start, end in matches:
            rule_id = nlp.vocab.strings[match_id]
            span = doc[start : end]
            allmatch.append(span)
        matchnu=len(allmatch)
        allmatchnu.append(matchnu)
    return allmatchnu

In [38]:
file=pd.read_csv('Bert_NIH_task4_Q1_LSTM.csv')
file=file.drop(columns=['Unnamed: 0'])
file=file.merge(allsent)


In [91]:
alllabel1=pd.DataFrame()
for i in tqdm(range(len(keylist))) :
    newdata=file[['newpid',keylist[i],'sent','pid']].sort_values(by=keylist[i],ascending =False)
    matchers=matcher(nlp, valuelist[i])
    pidlist=newdata.newpid.values  
    allmatchnu=reportfu(newdata, pidlist,matchers,nlp)
    allmatchnu1=np.where(np.array(allmatchnu) >0, 1, 0)
    newdata['label']=allmatchnu1
    newdata=newdata.rename(columns={keylist[i]:'predict_prob'})
    newdata['category']=[keylist[i]]*len(newdata)
    alllabel1=pd.concat([newdata,alllabel1])

100%|██████████| 8/8 [1:06:40<00:00, 500.11s/it]


In [98]:
gold=pd.read_csv('./inhibitor_NIH_ab_COVID.csv')

In [119]:
refg=alllabel1[alllabel1.pid.isin(gold.pid.values)]
refg=refg.sort_values(['predict_prob', 'label'], ascending=[False, True])
refg=refg[(refg.predict_prob>=0.7)&(refg.label==1)]

In [122]:
refg=refg[['newpid','sent','category']].drop_duplicates()
cand=alllabel1[['newpid','sent','category']].drop_duplicates()

In [123]:
categorylist=list(cand.category.drop_duplicates())

In [143]:
allframe=pd.DataFrame()
for i in tqdm(range(len(categorylist))):
    refg1=refg[refg.category==categorylist[i]]
    refgsent=list(refg1.sent.values)
    refgsentn=len(refgsent)
    cand1=cand[cand.category==categorylist[i]]
    candsent=list(cand1.sent.values)
    candsentn=len(candsent)
    if refgsentn==0:
        sentence=candsent
        score1=[0]*candsentn
        category=[categorylist[i]]*candsentn
        tempframe=pd.DataFrame(sentence,columns=['sent'])
        tempframe['score']=score1
        tempframe['category']=category
        allframe=pd.concat([allframe,tempframe])
    else :
        testrefsent=[]
        testcandsent=[]
        for j in range(len(candsent)):
            for k in range(len(refgsent)):
                testrefsent.append(refgsent[k])
                testcandsent.append(candsent[j])
        P, R, F1 = score(testcandsent, testrefsent, lang='en-sci', verbose=False)
        sentence=testcandsent
        score1=list(F1.numpy())
        tempframe=pd.DataFrame(sentence,columns=['sent'])
        tempframe['score']=score1
        tempframe=tempframe.groupby('sent').mean().reset_index()
        category=[categorylist[i]]*len(tempframe)
        tempframe['category']=category
        allframe=pd.concat([allframe,tempframe])
        
allframe=allframe.sort_values(by=['category','score'],ascending=False).drop_duplicates()    

100%|██████████| 8/8 [11:36<00:00, 87.01s/it] 


In [144]:
allframe1=allframe.merge(alllabel1)
allframe1=allframe1[['newpid','pid','category','score','predict_prob','label']]
allframe1=allframe1.sort_values(['score','label','predict_prob' ], ascending=[False,False, False])

In [173]:
goldframe=pd.DataFrame()
for i in categorylist:
    tempframe1=allframe1[allframe1.category==i].head(n=50)    
    tempframe1=tempframe1[tempframe1.label==1]
    tempframe1['real_label']=[1]*len(tempframe1)
    tempframe2=allframe1[allframe1.category==i].tail(n=50)    
    tempframe2=tempframe2[tempframe2.label==0]
    tempframe2['real_label']=[0]*len(tempframe2)
    tempframe=pd.concat([tempframe1,tempframe2])
    goldframe=pd.concat([goldframe,tempframe])

In [258]:
allframe2=pd.DataFrame()
for i in categorylist:
    tempframe=allframe1[allframe1.category==i]
    goldframe1=goldframe[goldframe.category==i]
    allx=tempframe[['score','predict_prob','label']].values
    goldx=goldframe1[['score','predict_prob','label']].values
    goldy=goldframe1[['real_label']].values
    fit=SGDRegressor()
    fit.fit(X=goldx ,y=goldy)
    ally=fit.predict(allx)
    tempframe['newscore']=ally
    tempframe=tempframe.sort_values(by='newscore',ascending=False)
    allframe2=pd.concat([tempframe,allframe2])
allframe2=allframe2[['newpid','pid','category','newscore']]

In [262]:
alllabel12=alllabel1[['pid','newpid','category','sent']]
alllabel12=alllabel12.merge(allframe2).sort_values(by=['category','newscore'],ascending=False)


In [266]:
alllabel12.to_csv('ALLRanking_TASK4_Q1.csv')

In [267]:
alllabel13=pd.DataFrame()
for i in categorylist:
    tempframe=alllabel12[alllabel12.category==i][0:1000]
    alllabel13=pd.concat([tempframe,alllabel13])
alllabel13.to_csv('ALLRanking_TASK4_Q1_TOP_1000.csv')

In [ ]:
allfile.to_csv('ALLRanking')